In [1]:
print("Kernel is running")

Kernel is running


In [2]:
from textblob import TextBlob
def correct_sentence_spelling(sentence):
    
    sentence = TextBlob(sentence)
    
    result = sentence.correct()
    
    return(result)

import pandas as pd
def file_to_list(file):
    rtn: object = []
    file_object: object = open(file, "r",encoding="utf-8")
    rtn: object = file_object.read().splitlines()
    file_object.close()
    return list(filter(None, pd.unique(rtn).tolist())) # Remove Empty/Duplicates Values
    pass

# Example #    
data_from_file: object = file_to_list('answer1.txt')
    
#sentence = "My naem is oeter"
for i in range(len(data_from_file)):
    data_from_file[i] = correct_sentence_spelling(data_from_file[i])
    print(data_from_file[i])
    
print(data_from_file)

Natural Language Processing, or NLP for short, is broadly defined as the automatic manipulation of natural language, like speech and text, by software.The study of natural language processing has been around for more than 50 years and grew out of the field of linguistic with the rise of computers.In this post, you will discover what natural language processing is and why it is so important.
Linguistic is the scientific study of language, including its grammar, semantic, and phonetics.
Classical linguistic involved devising and evaluation rules of language. Great progress was made on formal methods for santa and semantic, but for the most part, the interesting problems in natural language understanding resist clean mathematical formalisms.Broadly, a linguistic is anyone who studies language, but perhaps more colloquially, a self-defining linguistic may be more focused on being out in the field.Mathematics is the tool of science. Mathematicians working on natural language may refer to th

In [3]:
with open("spell_answer1.txt", "w") as txt_file:
    for line in data_from_file:
        txt_file.write("".join(line))

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize text
    #print("Summarize Text: \n", ". ".join(summarize_text))
    with open("summarize_answer1.txt", "w") as txt_file:
        txt_file.write(". ".join(summarize_text))

generate_summary( "spell_answer1.txt", 2)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [5]:
import numpy as np


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results

#########################

from gensim.models.keyedvectors import KeyedVectors
#from DocSim import DocSim

# Using the pre-trained word2vec model trained using Google news corpus of 3 billion running words.
# The model can be downloaded here: https://bit.ly/w2vgdrive (~1.4GB)
# Feel free to use to your own model.
googlenews_model_path = './data/GoogleNews-vectors-negative300.bin'
stopwords_path = "./data/stopwords_en.txt"

model = KeyedVectors.load_word2vec_format(googlenews_model_path, binary=True)
with open(stopwords_path, 'r') as fh:
    stopwords = fh.read().split(",")
ds = DocSim(model,stopwords=stopwords)



#################################  original answer to string
with open ("original_solution.txt", "r") as myfile:
    original_ans=myfile.readlines()

#################################
#source_doc = "how to delete an invoice"
source_doc = original_ans[0]
#print(source_doc)

################################# target answers to strings
with open ("summarize_answer1.txt", "r") as myfile:
    target_ans=myfile.readlines()

target_docs= []
#################################

#target_docs = ['delete a invoice', 'how do i remove an invoice', "purge an invoice"]
#target_docs.append(target_ans)
target_docs = target_ans
#print(target_docs)

sim_scores = ds.calculate_similarity(source_doc, target_docs)

print("Similarity Matrix Score:",sim_scores[0].get('score'))

########################
'''
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from DocSim import DocSim
import unittest


class DocSimTest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        test_model_path = './data/test_data.txt'
        cls.w2v_model = KeyedVectors.load_word2vec_format(test_model_path, binary=False)
        cls.stopwords = ['to', 'an', 'a']
        cls.doc_sim = DocSim(cls.w2v_model, cls.stopwords)

    def test_vectorize_with_valid_words(self):
        source_doc = 'how to delete an invoice'
        # same values dummy data will output same mean value
        expected = np.array([0.5, 0.5, 0.6, 0.3, 0.2, 0.1, 0.4, 0.6, 0.5, 0.5])
        actual = self.doc_sim.vectorize(source_doc)
        self.assertEqual(expected.all(), actual.all())


if __name__ == "__main__":
    unittest.main()
'''

Similarity Matrix Score: 0.8312151


'\nfrom gensim.models.keyedvectors import KeyedVectors\nimport numpy as np\nfrom DocSim import DocSim\nimport unittest\n\n\nclass DocSimTest(unittest.TestCase):\n    @classmethod\n    def setUpClass(cls):\n        test_model_path = \'./data/test_data.txt\'\n        cls.w2v_model = KeyedVectors.load_word2vec_format(test_model_path, binary=False)\n        cls.stopwords = [\'to\', \'an\', \'a\']\n        cls.doc_sim = DocSim(cls.w2v_model, cls.stopwords)\n\n    def test_vectorize_with_valid_words(self):\n        source_doc = \'how to delete an invoice\'\n        # same values dummy data will output same mean value\n        expected = np.array([0.5, 0.5, 0.6, 0.3, 0.2, 0.1, 0.4, 0.6, 0.5, 0.5])\n        actual = self.doc_sim.vectorize(source_doc)\n        self.assertEqual(expected.all(), actual.all())\n\n\nif __name__ == "__main__":\n    unittest.main()\n'

In [6]:
import math
print("marks scored by the student out of 10:-",math.ceil(10*sim_scores[0].get('score')))

marks scored by the student out of 10:- 9
